# Handbook of Solutions: Chapter 5 Exercises (Abraham-Silberschatz OS Concepts 10th Ed.)

---

## **Exercise 5.11: Context Switches for I/O-Bound vs. CPU-Bound Programs**

### **Question**
Of these two types of programs:
a. I/O-bound
b. CPU-bound

which is more likely to have voluntary context switches, and which is more likely to have nonvoluntary context switches? Explain your answer.

### **Answer**
*   **I/O-bound programs** are more likely to have **voluntary context switches**.
*   **CPU-bound programs** are more likely to have **nonvoluntary context switches**.

### **Detailed Explanation**

A **context switch** is the process of saving the state of one process and loading the saved state of another. They are categorized by their cause:

| Type | Cause | How it Happens |
| :--- | :--- | :--- |
| **Voluntary** | The running process **gives up the CPU willingly**. | Process issues a system call (e.g., for I/O, `wait()`, `yield()`). |
| **Nonvoluntary (Preemptive)** | The OS **forcibly removes the process** from the CPU. | Interrupt occurs (timer expiration, higher-priority process arrival). |

**I/O-Bound Programs:**
*   **Characteristic:** Frequent, short CPU bursts followed by I/O operations (e.g., reading from disk, waiting for user input).
*   **Behavior:** After a short CPU burst, the program will issue an I/O request (like `read()`). This system call triggers a transition to the **waiting** state, causing a **voluntary context switch**.
*   **Conclusion:** Their execution pattern naturally leads to many voluntary context switches.

**CPU-Bound Programs:**
*   **Characteristic:** Long CPU bursts with infrequent I/O operations (e.g., scientific calculations, matrix multiplication).
*   **Behavior:** They rarely issue I/O requests and thus rarely give up the CPU voluntarily. To prevent them from monopolizing the CPU and ensure fairness, the OS relies on **timer interrupts**. When the time quantum (in RR) expires or a higher-priority process arrives, the OS performs a **nonvoluntary context switch**.
*   **Conclusion:** They are primarily preempted by the OS scheduler, leading to nonvoluntary context switches.

---

## **Exercise 5.12: Conflicting Scheduling Criteria**

### **Question**
Discuss how the following pairs of scheduling criteria conflict in certain settings.

### **Answer & Explanation**

Scheduling algorithm design involves trade-offs. Optimizing for one metric often degrades another.

#### **a. CPU Utilization and Response Time**
*   **Conflict:** Maximizing CPU utilization can degrade response time, and vice-versa.
*   **Explanation:**
    *   **High CPU Utilization** is achieved by keeping the CPU busy at all times, often by favoring **CPU-bound processes** with long bursts or by minimizing context switches.
    *   **Good Response Time** (important for interactive systems) requires the CPU to be frequently available to handle short, incoming requests. This often means **preempting** long-running CPU-bound tasks to service interactive tasks, which introduces context switches and can leave the CPU idle briefly while switching.
    *   **Example:** In a **batch processing system** (goal: high utilization), using FCFS might keep the CPU 100% busy but lead to terrible response times for short interactive tasks. In a **time-sharing system** (goal: good response time), using RR with a small time quantum improves responsiveness but increases context switch overhead, potentially lowering overall CPU utilization.

#### **b. Average Turnaround Time and Maximum Waiting Time**
*   **Conflict:** Minimizing the average turnaround time can lead to an unacceptably high maximum waiting time for some processes (starvation).
*   **Explanation:**
    *   **Minimizing Average Turnaround Time** is the goal of algorithms like **SJF/SRTF**. They prioritize short jobs, which reduces the average.
    *   However, this can cause **indefinite postponement (starvation)** of long jobs. If short jobs keep arriving, a long job may wait indefinitely, causing its waiting time—and the system's **maximum waiting time**—to grow unbounded.
    *   **Example:** In a pure nonpreemptive SJF schedule, a single very long job will always be passed over if shorter jobs continue to arrive, maximizing its individual waiting time.

#### **c. I/O Device Utilization and CPU Utilization**
*   **Conflict:** Over-optimizing for one can lead to under-utilization of the other.
*   **Explanation:**
    *   **High CPU Utilization** is achieved by running CPU-bound tasks that rarely perform I/O.
    *   **High I/O Device Utilization** is achieved by keeping I/O devices busy, which requires having multiple I/O-bound processes that frequently issue I/O requests.
    *   The conflict arises in finding a balance. A system full of CPU-bound processes will have high CPU utilization but low I/O utilization (devices sit idle). A system full of I/O-bound processes will keep devices busy but may leave the CPU underutilized because processes are often blocked waiting for I/O.
    *   The ideal is a **mixed workload** where when one process is using the CPU, another is using an I/O device, allowing both to be utilized concurrently. However, scheduling to achieve perfect overlap is complex and these metrics can directly conflict if the workload is unbalanced.

---

## **Exercise 5.13: Implementing Priority in Lottery Scheduling**

### **Question**
In lottery scheduling, processes hold lottery tickets for CPU allocation. The BTV OS holds 50 lotteries/second, with each winner getting 20 ms. Describe how the BTV scheduler can ensure that higher-priority threads receive more attention from the CPU than lower-priority threads.

### **Answer**
The scheduler can assign tickets to threads **in proportion to their priority**. Higher-priority threads receive more tickets, increasing their probability of winning each lottery.

### **Detailed Explanation**

Lottery scheduling is a **probabilistic** algorithm that provides proportional-share resource allocation. The key mechanism is the distribution of tickets.

1.  **Assigning Tickets:** Each thread is assigned a number of lottery tickets. This number is not fixed; it is determined by the thread's **priority**. For example:
    *   Priority 10 (High): 100 tickets
    *   Priority 5 (Medium): 50 tickets
    *   Priority 1 (Low): 10 tickets
    The total tickets in the system = sum of all threads' tickets.

2.  **Holding the Lottery:** When a scheduling decision is needed (50 times per second in BTV), the scheduler picks a random number corresponding to a ticket. The thread holding that winning ticket runs for the next 20 ms time slice.

3.  **Mathematical Guarantee:** A thread's probability of winning the lottery (and thus receiving CPU time) is:
    $$
    P(\text{thread}) = \frac{\text{Tickets held by thread}}{\text{Total tickets in system}}
    $$
    Therefore, a thread with twice as many tickets as another has **twice the probability** of being scheduled, ensuring it receives "more attention" over time.

4.  **Dynamic Adjustment:** The system can dynamically adjust ticket allocations based on changing priorities or to enforce fairness, providing a flexible mechanism for priority-based scheduling without the starvation risks of strict priority scheduling.

---

## **Exercise 5.14: Run Queue Design for Multicore Systems**

### **Question**
On multicore systems, there are two general options for the run queue: (1) per-core run queues, or (2) a single shared run queue. What are the advantages and disadvantages of each?

### **Answer**

| Approach | Advantages | Disadvantages |
| :--- | :--- | :--- |
| **Per-Core Run Queues** (Each core has its own private queue) | **1. Scalability:** No locking contention for the queue, allowing cores to schedule independently. <br> **2. Cache Locality:** A process tends to run on the same core, making better use of cached data. <br> **3. Reduced Overhead:** No need for complex locking mechanisms for queue access. | **1. Load Imbalance:** One core may be idle while another has a long queue. Requires a **load-balancing** mechanism. <br> **2. Complexity of Migration:** Moving a process from one core's queue to another's (for load balancing) is costly and can break cache locality. |
| **Single Shared Run Queue** (One global queue for all cores) | **1. Automatic Load Balancing:** Cores automatically take the next available process, naturally balancing load. <br> **2. Simplicity:** Conceptually simple, easy to implement fairness policies (e.g., global RR). | **1. Scalability Bottleneck:** The single queue becomes a **contended shared resource**. Cores must lock the queue to access it, which can serialize scheduling decisions and hurt performance as core count increases. <br> **2. Poor Cache Locality:** A process may execute on a different core each time it is scheduled, causing **cache misses**. |

**Conclusion:** Modern OSes typically use a **hybrid approach** (e.g., per-core queues with periodic load balancing) to mitigate the disadvantages of each pure design.

---

## **Exercise 5.15: Implications of Parameters in Exponential Averaging**

### **Question**
Consider the exponential average formula used to predict the length of the next CPU burst:
$$
\tau_{n+1} = \alpha t_n + (1 - \alpha) \tau_n
$$
Where:
*   \( \tau_{n+1} \) = predicted length of next CPU burst
*   \( t_n \) = actual length of the \(n\)th CPU burst
*   \( \tau_n \) = previous prediction
*   \( \alpha \) = weighting parameter (\(0 \leq \alpha \leq 1\))

What are the implications of assigning the following values?

### **Answer & Explanation**

The parameter \( \alpha \) controls the weight given to the **most recent observation** versus the **entire past history**.

#### **a. α = 0 and τ₀ = 100 milliseconds**
*   **Implication:** **The prediction never changes. History is everything; recent behavior is ignored.**
*   **Explanation:**
    *   Substituting \( \alpha = 0 \) into the formula: \( \tau_{n+1} = 0 \cdot t_n + (1-0) \cdot \tau_n = \tau_n \).
    *   The new prediction is always equal to the old prediction, regardless of the actual burst length \( t_n \).
    *   The system will **always use the initial guess \( \tau_0 = 100 \)** for all future predictions. This eliminates the adaptive nature of the algorithm, making it useless for tracking changes in process behavior.

#### **b. α = 0.99 and τ₀ = 10 milliseconds**
*   **Implication:** **The prediction is overwhelmingly based on the most recent burst. Past history is nearly irrelevant.**
*   **Explanation:**
    *   Substituting \( \alpha = 0.99 \): \( \tau_{n+1} = 0.99 \cdot t_n + 0.01 \cdot \tau_n \).
    *   The new prediction is 99% determined by the actual length of the last CPU burst (\( t_n \)) and only 1% by the entire previous history (\( \tau_n \)).
    *   The algorithm becomes **extremely responsive to recent changes** but also highly volatile. A single anomalous long or short burst will drastically alter the prediction. The initial guess \( \tau_0 = 10 \) becomes irrelevant after just one or two bursts.
    *   This setting is useful if a process's burst lengths are **highly variable** and you want the predictor to quickly adapt to a new pattern. However, it may overreact to noise.

**General Insight:** Choosing \( \alpha \) is a trade-off between **stability** (low \( \alpha \), smooth predictions) and **responsiveness** (high \( \alpha \), quick adaptation). A typical value like \( \alpha = 0.5 \) provides a balance.



## **Exercise 5.16: Regressive Round-Robin Scheduler**

### **Question**
A variation of round-robin called **regressive round-robin** assigns each process a time quantum and priority. Initial time quantum = 50 ms. Rules:
1. If a process uses its **entire quantum** (doesn't block for I/O): 
   - Add 10 ms to its quantum (max 100 ms)
   - Boost its priority
2. If a process **blocks before quantum ends**:
   - Reduce quantum by 5 ms
   - Priority unchanged

Which process type (CPU-bound or I/O-bound) does this scheduler favor? Explain.

### **Answer**
This scheduler **favors I/O-bound processes**.

### **Detailed Explanation**
The scheduler's rules create feedback that rewards I/O-bound behavior and penalizes CPU-bound behavior:

| Process Type | Typical Behavior | Effect under Regressive RR | Consequence |
| :--- | :--- | :--- | :--- |
| **I/O-Bound** | Frequently blocks for I/O before quantum expires. | Quantum decreases by 5 ms each schedule, but **priority remains same**. | While quantum shrinks, the process maintains its priority level and actually gets scheduled more **frequently** (due to shorter bursts), leading to better responsiveness. |
| **CPU-Bound** | Rarely blocks; uses full quantum. | Quantum increases (up to 100 ms), but **priority is boosted** (likely meaning lowered in Linux-style where higher number = lower priority). | Gets longer quanta but at **lower priority**. This means it will be scheduled less often than higher-priority I/O-bound processes. |

**Key Insight:** The scheduler achieves **priority inversion** relative to standard RR:
- **I/O-bound processes** keep their original (higher) priority and get shorter quanta → they finish their CPU work quickly and return to I/O, getting frequent, responsive service.
- **CPU-bound processes** get lower priority and longer quanta → they run for longer periods but less frequently, preventing them from monopolizing the CPU.

This design explicitly favors **interactive/I/O-bound processes** by ensuring they don't get stuck behind long CPU bursts, improving overall system responsiveness.

---

## **Exercise 5.17: Comprehensive Scheduling Comparison (Second Set)**

### **Question**
Consider the following processes (all arriving at time 0):

| Process | Burst Time | Priority |
| :--- | :---: | :---: |
| P1 | 5 | 4 |
| P2 | 3 | 1 |
| P3 | 1 | 2 |
| P4 | 7 | 2 |
| P5 | 4 | 3 |

*(Higher priority number = higher priority)*

**Tasks:**
a. Draw Gantt charts for: FCFS, SJF, Nonpreemptive Priority, RR (quantum=2).
b. Calculate turnaround time for each process under each algorithm.
c. Calculate waiting time for each process.
d. Identify which algorithm gives minimum average waiting time.

### **Answer**

#### **a. Gantt Charts**

1. **FCFS (First-Come, First-Served)**
   ```
   | P1 | P2 | P3 | P4  | P5   |
   0    5    8    9     16    20
   ```

2. **SJF (Shortest-Job-First) - Nonpreemptive**
   - Order by burst time: P3(1), P2(3), P5(4), P1(5), P4(7)
   ```
   | P3 | P2 | P5 | P1 | P4  |
   0    1    4    8   13    20
   ```

3. **Nonpreemptive Priority** (Higher number = higher priority)
   - Priorities: P1(4), P2(1), P3(2), P4(2), P5(3)
   - Order by priority: P1(4), P5(3), then P3 & P4 (both 2, tie-break by arrival → P3 then P4), then P2(1)
   ```
   | P1 | P5 | P3 | P4  | P2 |
   0    5    9   10    17   20
   ```

4. **RR (Round Robin, Quantum = 2)**
   - All processes arrive at time 0, order: P1, P2, P3, P4, P5
   ```
   |P1|P2|P3|P4|P5|P1|P2|P4|P5|P1|P4|P5|P4|
   0 2 4 5 6 8 10 11 13 14 16 18 19 20
   ```
   **Detailed RR Sequence:**
   - Time 0-2: P1 (3 left)
   - 2-4: P2 (1 left)
   - 4-5: P3 (1 burst, finishes at 5)
   - 5-6: P4 (5 left)
   - 6-8: P5 (2 left)
   - 8-10: P1 (1 left)
   - 10-11: P2 (finishes at 11)
   - 11-13: P4 (3 left)
   - 13-14: P5 (finishes at 14)
   - 14-16: P1 (finishes at 16)
   - 16-18: P4 (1 left)
   - 18-19: P4 (finishes at 19)
   - 19-20: [CPU idle? Actually P4 finished at 19, so idle from 19-20]

#### **b. Turnaround Time (TAT) Table**
*TAT = Completion Time - Arrival Time (Arrival=0 for all)*

| Process | FCFS | SJF | Priority | RR |
| :--- | :---: | :---: | :---: | :---: |
| P1 | 5 | 13 | 5 | 16 |
| P2 | 8 | 4 | 20 | 11 |
| P3 | 9 | 1 | 10 | 5 |
| P4 | 16 | 20 | 17 | 19 |
| P5 | 20 | 8 | 9 | 14 |

#### **c. Waiting Time (WT) Table**
*WT = Turnaround Time - Burst Time*

| Process | Burst | FCFS | SJF | Priority | RR |
| :--- | :---: | :---: | :---: | :---: | :---: |
| P1 | 5 | 0 | 8 | 0 | 11 |
| P2 | 3 | 5 | 1 | 17 | 8 |
| P3 | 1 | 8 | 0 | 9 | 4 |
| P4 | 7 | 9 | 13 | 10 | 12 |
| P5 | 4 | 16 | 4 | 5 | 10 |

#### **d. Algorithm with Minimum Average Waiting Time**
Calculate averages from part (c):

- **FCFS:** (0+5+8+9+16)/5 = 38/5 = **7.6**
- **SJF:** (8+1+0+13+4)/5 = 26/5 = **5.2**
- **Priority:** (0+17+9+10+5)/5 = 41/5 = **8.2**
- **RR:** (11+8+4+12+10)/5 = 45/5 = **9.0**

**SJF** gives the **minimum average waiting time (5.2)**, which is expected as SJF is optimal for minimizing average waiting time when all processes arrive simultaneously.

---

## **Exercise 5.19: The `nice` Command and Privileges**

### **Question**
The `nice` command sets the nice value of a process on Linux/UNIX. Why do some systems allow any user to set nice value ≥ 0, but only root can set values < 0?

### **Answer**
This design prevents **priority inversion attacks** and ensures system stability by preventing non-privileged users from monopolizing CPU resources.

### **Detailed Explanation**
In UNIX/Linux, the **nice value** ranges from -20 (highest priority) to +19 (lowest priority). The actual priority is calculated as: `Priority = Base + nice`, where lower numerical priority value means higher scheduling priority in the kernel.

| Action | Effect | Privilege Required | Rationale |
| :--- | :--- | :--- | :--- |
| **Increase nice value (≥ 0)** | Lowers process priority (makes it "nicer" to others) | Any user | This is **safe**—it only reduces a process's share of CPU, potentially slowing it down but not harming other processes or system stability. |
| **Decrease nice value (< 0)** | Raises process priority (makes it more aggressive) | Only root (superuser) | This is **dangerous**—a user could make their process higher priority than system daemons or other users' processes, causing **starvation** or system unresponsiveness. |

**Security & Fairness Implications:**
1. **Denial-of-Service Prevention:** If any user could set high priority (negative nice), they could monopolize CPU and starve critical system processes.
2. **System Integrity:** Kernel and administrative processes need guaranteed CPU access for system maintenance, I/O handling, and security monitoring.
3. **Fair Sharing:** In multi-user systems, the root administrator must arbitrate resource contention to ensure fair allocation among users.

This privilege separation is a classic example of the **principle of least privilege** in OS design.

---

## **Exercise 5.20: Scheduling Algorithms and Starvation**

### **Question**
Which of the following scheduling algorithms could result in starvation?
a. First-come, first-served
b. Shortest job first
c. Round robin
d. Priority

### **Answer**
Algorithms that could result in starvation: **b. Shortest job first** and **d. Priority**.

### **Detailed Explanation**

**Starvation** occurs when a process is indefinitely denied CPU time because other processes are always preferred.

| Algorithm | Starvation Possible? | Why |
| :--- | :--- | :--- |
| **FCFS** | **No** | Every process eventually gets the CPU in arrival order. No process is skipped indefinitely. |
| **SJF** | **Yes** | A long job may wait forever if shorter jobs keep arriving. This is the classic starvation scenario for SJF. |
| **Round Robin** | **No** | Each process gets a time slice periodically. No process waits more than (n-1)*q time units for n processes with quantum q. |
| **Priority** | **Yes** | Low-priority processes may never run if higher-priority processes continuously arrive. This is common in pure priority scheduling. |

**Mitigations:**
- **Aging** is often used with SJF and Priority schedulers: gradually increasing priority of waiting processes to eventually break starvation.
- **Multilevel feedback queues** combine RR at different priority levels with process migration between levels to prevent indefinite starvation.


## **Exercise 5.21: Variant of RR with Pointers to PCBs**

### **Question**
Consider a variant of the RR scheduling algorithm in which the entries in the ready queue are pointers to the PCBs.

a. What would be the effect of putting two pointers to the same process in the ready queue?
b. What would be two major advantages and two disadvantages of this scheme?
c. How would you modify the basic RR algorithm to achieve the same effect without the duplicate pointers?

### **Answer**

#### **a. Effect of Duplicate Pointers**
Placing two pointers to the same process in the ready queue would cause that process to be **selected twice as often** in the round-robin cycle. It would receive approximately twice the CPU time compared to processes with only one pointer, effectively increasing its priority or weight.

#### **b. Advantages and Disadvantages**
**Advantages:**
1. **Simple Priority Mechanism:** Provides an easy way to implement **weighted fair sharing** without complex priority scheduling algorithms.
2. **Flexibility:** The scheduler can dynamically adjust the relative CPU allocation by adding or removing pointers.

**Disadvantages:**
1. **Starvation Risk:** If some processes have many pointers and others few, it can lead to unfairness or starvation.
2. **Management Overhead:** When a process blocks or terminates, all its pointers must be located and removed from the queue, requiring additional operations.

#### **c. Modification without Duplicate Pointers**
To achieve proportional CPU allocation without duplicate pointers, modify RR to use **variable time quanta** or **weighted round-robin**:
- Assign each process a **weight** (e.g., 2 for processes that would have had two pointers).
- Allocate CPU time proportional to weights, either by adjusting time quantum sizes or by scheduling processes with frequency proportional to their weights.

---

## **Exercise 5.22: CPU Utilization with Mixed Workload**

### **Question**
Consider a system running ten I/O-bound tasks and one CPU-bound task. Assume that the I/O-bound tasks issue an I/O operation once for every millisecond of CPU computing and that each I/O operation takes 10 milliseconds to complete. Also assume that the context-switching overhead is 0.1 millisecond and that all processes are long-running tasks. Describe the CPU utilization for a round-robin scheduler when:

a. The time quantum is 1 millisecond
b. The time quantum is 10 milliseconds

### **Answer**
We define **CPU utilization** as the percentage of time the CPU is executing user processes (excluding context-switch overhead).

**Given:**
- 10 I/O-bound tasks: CPU burst = 1 ms, I/O time = 10 ms.
- 1 CPU-bound task: CPU burst = entire quantum (never blocks).
- Context switch time = 0.1 ms.
- All tasks are long-running (always have work to do).

#### **a. Time Quantum = 1 ms**
Each I/O-bound task runs for 1 ms, then blocks for I/O. The CPU-bound task runs for 1 ms per quantum.

**Cycle Analysis:**
In a cycle where all 11 tasks run once:
- Total CPU useful time = 10 × 1 ms (I/O tasks) + 1 × 1 ms (CPU task) = 11 ms.
- Number of context switches = 11 (after each task).
- Total context switch time = 11 × 0.1 ms = 1.1 ms.
- Total cycle time = 11 ms + 1.1 ms = 12.1 ms.

\[
\text{CPU utilization} = \frac{\text{Useful CPU time}}{\text{Total time}} = \frac{11}{12.1} \approx 0.909 = 90.9\%
\]

#### **b. Time Quantum = 10 ms**
I/O-bound tasks still run for only 1 ms then block (they don't use full quantum). CPU-bound task runs for up to 10 ms.

**Cycle Analysis:**
Consider a period where all I/O tasks run once and the CPU task runs once:
- Useful CPU time = 10 × 1 ms + 10 ms = 20 ms.
- Number of context switches = 11 (10 after I/O tasks block, 1 after CPU task's quantum expires).
- Total context switch time = 11 × 0.1 ms = 1.1 ms.
- Total time = 20 ms + 1.1 ms = 21.1 ms.

\[
\text{CPU utilization} = \frac{20}{21.1} \approx 0.948 = 94.8\%
\]

**Conclusion:** A larger time quantum reduces context switch overhead and improves CPU utilization for this mixed workload.

---

## **Exercise 5.23: Maximizing CPU Time in Multilevel Queue Scheduling**

### **Question**
Consider a system implementing multilevel queue scheduling. What strategy can a computer user employ to maximize the amount of CPU time allocated to the user’s process?

### **Answer**
A user can structure their process to **mimic an interactive (I/O-bound) process** to be placed in a higher-priority queue.

### **Detailed Explanation**
Multilevel queue schedulers typically have:
- **High-priority queues** for interactive processes (short time quanta, frequent scheduling).
- **Low-priority queues** for batch/CPU-bound processes (long time quanta, infrequent scheduling).

**Strategies:**
1. **Frequent I/O Operations:** Insert periodic I/O calls (e.g., small writes, sleep calls) to make the process appear I/O-bound, favoring placement in a higher-priority queue.
2. **Exploit Priority Rules:** If the system allows user-set priorities (e.g., `nice` values), set the process to highest allowed priority.
3. **Process Characteristics:** Design the program to have short CPU bursts (< time quantum) to avoid being demoted to lower-priority queues in a **multilevel feedback queue** system.

---

## **Exercise 5.24: Dynamic Priority Scheduling**

### **Question**
Consider a preemptive priority scheduling algorithm based on dynamically changing priorities. Larger priority numbers imply higher priority. When a process is waiting for the CPU (in the ready queue, but not running), its priority changes at a rate α. When it is running, its priority changes at a rate β. All processes are given a priority of 0 when they enter the ready queue. The parameters α and β can be set to give many different scheduling algorithms.

a. What is the algorithm that results from β > α > 0?
b. What is the algorithm that results from α < β < 0?

### **Answer**

#### **a. β > α > 0 (Both positive, β larger)**
- Running processes' priorities increase faster than waiting processes'.
- Once a process starts running, it quickly becomes the highest priority and tends to keep the CPU.
- This approximates **First-Come, First-Served (FCFS)** or **nonpreemptive scheduling**, as the initial process to run will retain control.

#### **b. α < β < 0 (Both negative, α more negative)**
- Priorities decrease (since negative rates), but waiting processes' priorities drop faster.
- A running process retains a relatively higher priority compared to waiting processes.
- New processes (priority 0) have higher priority than older waiting processes (negative priorities), leading to **Last-Come, First-Served (LCFS)** behavior, possibly with starvation of older processes.

---

## **Exercise 5.25: Discrimination for Short Processes**

### **Question**
Explain how the following scheduling algorithms discriminate either in favor of or against short processes:

a. FCFS
b. RR
c. Multilevel feedback queues

### **Answer**

#### **a. FCFS (First-Come, First-Served)**
- **Discriminates against short processes.**
- **Why:** The **convoy effect**—a short process arriving after a long process must wait for the long process to complete, increasing its waiting time disproportionately.

#### **b. RR (Round Robin)**
- **Treats all processes equally, but short processes benefit.**
- **Why:** Each process gets equal time slices; short processes finish earlier because they require fewer total slices, reducing their turnaround time.

#### **c. Multilevel Feedback Queues (MFQ)**
- **Generally favors short (I/O-bound) processes.**
- **Why:** Processes with short CPU bursts are promoted to higher-priority queues, getting more frequent CPU access. Long CPU-bound processes are demoted to lower-priority queues.

---

## **Exercise 5.26: Shared Ready Queue in SMP**

### **Question**
Describe why a shared ready queue might suffer from performance problems in an SMP environment.

### **Answer**
A shared ready queue becomes a **contention bottleneck** due to:
1. **Lock Synchronization:** Multiple processors must lock the queue to enqueue/dequeue processes, causing serialization and spin-waiting.
2. **Cache Coherence Overhead:** Frequent updates to the shared queue cause cache invalidation across processors, increasing memory latency.
3. **Scalability Limit:** As processor count grows, contention increases, reducing parallel efficiency.

**Solution:** Use per-processor run queues with work-stealing or load-balancing.

---

## **Exercise 5.27: Load Balancing with Priority-Based Scheduling**

### **Question**
Consider a load-balancing algorithm that ensures that each queue has approximately the same number of threads, independent of priority. How effectively would a priority-based scheduling algorithm handle this situation if one run queue had all high-priority threads and a second queue had all low-priority threads?

### **Answer**
It would handle it **poorly**. The load balancer ignores priority, leading to:
- **Processor 1 (high-priority queue):** Responsive for high-priority threads.
- **Processor 2 (low-priority queue):** Busy with low-priority threads, while high-priority threads might be waiting if Processor 1 is overloaded.
- **Inefficiency:** Processor 2 could be executing low-priority threads while high-priority threads exist, violating priority-based scheduling goals.

**Solution:** Load balancing should consider both thread count and priority, distributing high-priority threads across processors.

---

## **Exercise 5.28: Process Placement in Per-Processor Run Queues**

### **Question**
Assume that an SMP system has private, per-processor run queues. When a new process is created, it can be placed in either the same queue as the parent process or a separate queue.

a. What are the benefits of placing the new process in the same queue as its parent?
b. What are the benefits of placing the new process in a different queue?

### **Answer**

#### **a. Same Queue as Parent**
- **Cache Affinity:** Parent and child likely share code/data; keeping them on the same processor improves cache hit rates.
- **Synchronization Efficiency:** If they communicate frequently, shared memory accesses are faster within the same processor’s cache.

#### **b. Different Queue**
- **Load Distribution:** Spreads work across processors, improving overall throughput.
- **Parallel Execution:** Allows true concurrency if parent and child are independent.
- **Fault Isolation:** A busy processor doesn’t delay both processes.

**Trade-off:** Systems often use **affinity scheduling** (same queue initially) with **migration** for load balancing.


## **Exercise 5.29: NUMA-Aware Scheduling**

### **Question**
Assume that a thread has blocked for network I/O and is eligible to run again. Describe why a NUMA-aware scheduling algorithm should reschedule the thread on the same CPU on which it previously ran.

### **Answer**
In a **NUMA (Non-Uniform Memory Access)** system, memory access time depends on the memory location relative to the processor. Each processor has **local memory** (fast access) and **remote memory** (slower access across interconnects).

**Reasons for Rescheduling on Same CPU:**

1. **Memory Locality:** The thread's memory pages were likely allocated in the **local memory** of the CPU it was running on. Rescheduling it on the same CPU ensures fast access to its working set.
2. **Cache Warmth:** The CPU's caches may still contain the thread's data and instructions, reducing cache miss penalties.
3. **Reduced NUMA Penalty:** If scheduled on a different CPU, the thread would experience:
   - **Remote memory accesses** (3-5x slower than local)
   - **Complete cache cold start** (all cache misses)
   - **Increased memory contention** on remote memory controllers

**Performance Impact:** In NUMA systems, incorrect thread placement can degrade performance by 30-50% due to remote memory access latency.

---

## **Exercise 5.30: Windows Thread Priority Calculation**

### **Question**
Using the Windows scheduling algorithm, determine the numeric priority of each of the following threads.

Windows uses:
- **Priority Classes** (base priorities): IDLE (4), BELOW_NORMAL (6), NORMAL (8), ABOVE_NORMAL (10), HIGH (13), REALTIME (24)
- **Relative Priorities** within class: IDLE (-2), BELOW_NORMAL (-1), NORMAL (0), ABOVE_NORMAL (+1), HIGHEST (+2), TIME_CRITICAL (+3 for non-realtime, +16 for realtime)

**Numeric Priority = Base(Priority Class) + Relative Priority**

### **Answer**

#### **a. REALTIME PRIORITY CLASS with NORMAL relative priority**
- REALTIME base = 24
- NORMAL relative = 0
- **Numeric Priority = 24 + 0 = 24**

#### **b. ABOVE NORMAL PRIORITY CLASS with HIGHEST relative priority**
- ABOVE_NORMAL base = 10
- HIGHEST relative = +2
- **Numeric Priority = 10 + 2 = 12**

#### **c. BELOW NORMAL PRIORITY CLASS with ABOVE NORMAL relative priority**
- BELOW_NORMAL base = 6
- ABOVE_NORMAL relative = +1
- **Numeric Priority = 6 + 1 = 7**

---

## **Exercise 5.31: Highest Possible Windows Priority**

### **Question**
Assuming that no threads belong to the REALTIME PRIORITY CLASS and that none may be assigned a TIME_CRITICAL priority, what combination of priority class and priority corresponds to the highest possible relative priority in Windows scheduling?

### **Answer**
**HIGH PRIORITY CLASS with HIGHEST relative priority**

**Calculation:**
- Without REALTIME class, the highest base priority is **HIGH** = 13
- Without TIME_CRITICAL, the highest relative priority is **HIGHEST** = +2
- **Numeric Priority = 13 + 2 = 15**

**Alternative expression:** Threads in the HIGH priority class with HIGHEST relative adjustment have the highest possible priority under these constraints.

---

## **Exercise 5.32: Solaris Time-Sharing Thread Scheduling**

### **Question**
Consider the scheduling algorithm in Solaris for time-sharing threads.

**Solaris Priority System:**
- Priorities 0-59, higher number = higher priority
- Time quantum = (priority ≤ 59) ? (20 + (priority/4)) ms : 200 ms
- Priority adjustments:
  - Uses entire quantum without blocking: priority decreases (penalized for being CPU-bound)
  - Blocks before quantum expires: priority increases (rewarded for being I/O-bound)

### **Answer**

#### **a. Time quantum for priority 15? Priority 40?**
- **Priority 15:** Quantum = 20 + ⌊15/4⌋ = 20 + 3 = **23 ms**
- **Priority 40:** Quantum = 20 + ⌊40/4⌋ = 20 + 10 = **30 ms**

#### **b. Thread with priority 50 uses entire quantum without blocking**
- Using entire quantum → **priority decreases**
- Typical decrease: by 1-5 priority points (implementation specific)
- **New priority likely in range 45-49** (probably 49 if decreasing by 1)

#### **c. Thread with priority 20 blocks for I/O before quantum expires**
- Blocking before quantum ends → **priority increases**
- Typical increase: by 1-10 priority points (more if blocked early)
- **New priority likely in range 21-30** (could be up to 30 for very early block)

**Design Philosophy:** This implements a **multilevel feedback queue** that rewards I/O-bound (interactive) threads and penalizes CPU-bound threads.

---

## **Exercise 5.33: Linux CFS Scheduler and vruntime**

### **Question**
Assume two tasks, A (nice = -5) and B (nice = +5), running on Linux with CFS scheduler. Describe how their vruntime values vary in these scenarios:

**CFS Basics:** 
- Each task has virtual runtime (vruntime) = actual runtime × weight factor
- Lower nice = higher weight = vruntime increases slower
- CFS always runs task with **lowest vruntime**

### **Answer**

#### **Scenario 1: Both A and B are CPU-bound**
- Both constantly use CPU, accumulate actual runtime
- A (nice = -5, higher priority) has **higher weight** → vruntime increases **slower** than B
- B (nice = +5, lower priority) has **lower weight** → vruntime increases **faster**
- **Result:** A's vruntime stays lower than B's, so A gets **more CPU time** (approximately 3:1 ratio due to weight differences)

#### **Scenario 2: A is I/O-bound, B is CPU-bound**
- A blocks frequently for I/O → accumulates little actual runtime → vruntime increases very slowly
- B runs continuously → accumulates actual runtime quickly → vruntime increases rapidly
- **Result:** A's vruntime stays very low, so when A becomes ready, it immediately preempts B. A gets excellent responsiveness.

#### **Scenario 3: A is CPU-bound, B is I/O-bound**
- A runs continuously → vruntime increases (but slower due to higher weight)
- B blocks frequently → vruntime increases very slowly when it runs
- **Result:** B's vruntime may become lower than A's when B wakes up, allowing B to preempt A. This ensures I/O-bound tasks remain responsive even with lower priority (nice +5).

**Key Insight:** CFS achieves fairness by **comparing vruntime, not raw CPU time**, allowing it to properly handle mixed workloads.

---

## **Exercise 5.34: Rate-Monotonic vs Earliest-Deadline-First**

### **Question**
Provide a specific circumstance that illustrates where rate-monotonic scheduling is inferior to earliest-deadline-first scheduling in meeting real-time process deadlines.

### **Answer**
**Scenario:**
- **Task T1:** Period = 50 ms, Execution time = 25 ms, Deadline = 50 ms
- **Task T2:** Period = 75 ms, Execution time = 30 ms, Deadline = 75 ms

**Rate-Monotonic (RM) Analysis:**
- Assigns higher priority to task with **shorter period** (T1)
- T1 always preempts T2
- Schedule T1 first (0-25), then T2 (25-55), but T1's next period starts at 50
- **Problem:** At time 50, T1 preempts T2 (which started at 25 and needs until 55)
- T1 runs 50-75, then T2 resumes 75-80 (but T2's deadline was 75!)
- **Result:** T2 **misses deadline** at 75

**Earliest-Deadline-First (EDF) Analysis:**
- Always runs task with **earliest deadline**
- Initially: T1 deadline = 50, T2 deadline = 75 → run T1 (0-25)
- At 25: T1 next deadline = 50, T2 deadline = 75 → still run T1? Wait, need to check.
- Actually, EDF would schedule: T1 (0-25), T2 (25-55), T1 (55-80)...
- **Deadline check:** T2 completes at 55 < deadline 75 ✓, T1 completes at 80 < deadline 100 ✓

**Conclusion:** RM fails this task set even though CPU utilization (25/50 + 30/75 = 0.5 + 0.4 = 0.9) is below the RM bound (~0.828 for 2 tasks). EDF successfully schedules it because EDF can achieve 100% utilization.

---

## **Exercise 5.35: Rate-Monotonic vs EDF Scheduling**

### **Question**
Consider two processes:
- P₁: period p₁ = 50, execution time t₁ = 25
- P₂: period p₂ = 75, execution time t₂ = 30

a. Can these be scheduled using rate-monotonic scheduling? Illustrate with Gantt chart.
b. Illustrate scheduling using earliest-deadline-first (EDF).

### **Answer**

#### **a. Rate-Monotonic Scheduling**
**Priority:** P₁ has higher priority (shorter period = 50)

**Gantt Chart (first 150 ms):**
```
Time:   0    25    50    75    100   125   150
        |-----|-----|-----|-----|-----|-----|
P1:     [====]     [====]     [====]     [====]
        0-25        50-75      100-125
P2:           [==============]           [====...
                25-55 (preempted!)          125-155
```
**Detailed Breakdown:**
- 0-25: P₁ runs (completes)
- 25-50: P₂ runs (needs 30 total, gets 25 so far)
- 50-55: P₁ preempts (higher priority), P₂ paused with 5 ms remaining
- 55-75: P₁ completes its 25 ms burst
- 75-80: P₂ resumes and completes its remaining 5 ms
- But P₂'s first deadline was at 75! It completed at 80 → **MISSED DEADLINE**

**Conclusion:** RM fails. Utilization = 25/50 + 30/75 = 0.5 + 0.4 = 0.9 > 0.828 (RM bound for 2 tasks), so guaranteed to fail.

#### **b. Earliest-Deadline-First Scheduling**
EDF always runs task with earliest absolute deadline.

**Gantt Chart (first 150 ms):**
```
Time:   0    25    50    75    100   125   150
        |-----|-----|-----|-----|-----|-----|
P1:     [====]           [====]     [====]
        0-25 (d=50)      55-80     105-130
                         (d=100)    (d=150)
P2:           [==============]     [======...
                25-55               80-110
                (d=75)              (d=150)
```
**Schedule:**
- 0-25: P₁ runs (deadline 50)
- 25-55: P₂ runs (deadline 75) - completes at 55 < 75 ✓
- 55-80: P₁ runs (next instance, deadline 100) - completes at 80 < 100 ✓
- 80-110: P₂ runs (next instance, deadline 150) - completes at 110 < 150 ✓
- 105-130: P₁ runs (skipped? Actually P₁ at 100... Let's recalculate carefully)

**Correct EDF Schedule:**
Time 0: P₁(d50), P₂(d75) → run P₁ (0-25)
Time 25: P₁ next d=100, P₂(d75) → run P₂ (25-55)
Time 55: P₁(d100), P₂ next d=150 → run P₁ (55-80)
Time 80: P₁ next d=150, P₂(d150) → tie, can run either (say P₂: 80-110)
Time 110: P₁(d150) → run P₁ (110-135)
All deadlines met ✓

---

## **Exercise 5.36: Bounded Latency in Hard Real-Time Systems**

### **Question**
Explain why interrupt and dispatch latency times must be bounded in a hard real-time system.

### **Answer**
In **hard real-time systems**, missing a deadline can cause **catastrophic failure** (e.g., aircraft control, medical devices, nuclear plant control).

**Two Critical Latencies:**

1. **Interrupt Latency:** Time from interrupt signal to start of ISR
   - Must be bounded to ensure timely response to external events
   - Factors: OS may disable interrupts (critical sections), interrupt controller delays

2. **Dispatch Latency:** Time from ISR completion to task start
   - Must be bounded to ensure high-priority tasks run promptly
   - Factors: Context switch time, scheduler decisions, cache flushing

**Why Bounded/Boundable:**
- **Predictability:** System must guarantee worst-case response time ≤ deadline
- **Schedulability Analysis:** Mathematical analysis (e.g., RM, EDF) requires known maximum latencies
- **Certification:** Safety-critical systems require formal verification of timing constraints

**Example:** In fly-by-wire systems, control loop must complete within 1-2 ms. Unbounded latency could cause oscillation or loss of control.

---

## **Exercise 5.37: Heterogeneous Multiprocessing in Mobile Systems**

### **Question**
Describe the advantages of using heterogeneous multiprocessing in a mobile system.

### **Answer**
**Heterogeneous Multiprocessing (HMP)** uses cores with different performance/power characteristics (e.g., ARM big.LITTLE: few "big" high-performance cores + many "LITTLE" power-efficient cores).

### **Advantages:**

1. **Energy Efficiency:** 
   - Light tasks → LITTLE cores (low power)
   - Heavy tasks → big cores (high performance)
   - Dynamic migration balances performance needs with battery life

2. **Thermal Management:**
   - Spread heat generation across different core types
   - Use LITTLE cores when thermal throttling required
   - Prevents overheating and performance degradation

3. **Performance Scaling:**
   - Fine-grained control over performance/power trade-off
   - Better than DVFS alone (dynamic voltage/frequency scaling)

4. **Cost Optimization:**
   - Mix of core types provides better performance/$ than homogeneous design
   - Smaller LITTLE cores occupy less die area

5. **Workload Matching:**
   - Background tasks (email sync, notifications) → LITTLE cores
   - Foreground tasks (gaming, camera) → big cores
   - OS scheduler intelligently places threads based on requirements

**Example:** Samsung Exynos/Qualcomm Snapdragon use HMP to achieve all-day battery life while providing burst performance when needed.
